In [1]:
import pandas as pd

#importo mi data con clusters

In [2]:
data_clusters = pd.read_csv('data_clusters.csv')
display(data_clusters.shape)
#data_clusters.head()

(34383, 3)

In [3]:
data_clusters.columns = ['Song_name', 'Artist_name', 'Clusters']

In [4]:
data_clusters.head()

,Song_name,Artist_name,Clusters
0,If I Ever Feel Better,Phoenix,11
1,Dance To The Underground,Radio 4,1
2,How We Do,Mount Sims,11
3,Fuck the Pain Away,Peaches,7
4,Street Surgery 2,Zongamin,6


#importo mi top 150

In [5]:
top_songs = pd.read_csv('top_songs.csv')
display(top_songs.shape)
#top_songs.head()

(150, 2)

In [6]:
top_songs.columns = ['Song_name', 'Artist_name']

In [17]:
top_songs.head()

,Song_name,Artist_name
0,lovin on me,Jack Harlow
1,lil boo thang,Paul Russell
2,i remember everything (feat. kacey musgraves),Zach Bryan
3,white horse,Chris Stapleton
4,save me (with lainey wilson),Jelly Roll


#Cargar modelo KMeans
#Cargar escalador de datos

In [8]:
import pickle

kmeans = pickle.load(open('kmean.pkl', 'rb'))
dataspotify_sca = pickle.load(open('scaler.pkl', 'rb'))


In [9]:
from pandas import json_normalize

In [10]:
secrets_file = open("secrets.txt","r")

In [11]:
string = secrets_file.read()

In [12]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

#creo mi funcion

In [15]:
import random
def recomendar(top_songs, cancion_ingresada, data_clusters):
    #Paso a minuscula el nombre de las canciones
    cancion_ingresada = cancion_ingresada.lower()
    top_songs['Song_name'] = top_songs['Song_name'].str.lower()
    data_clusters['Song_name'] = data_clusters['Song_name'].str.lower()
    #Chequeo si la cancion q ingrese esta en mi top
    if cancion_ingresada in top_songs['Song_name'].values:
        canciones_disponibles = top_songs[top_songs['Song_name'] != cancion_ingresada]
        cancion_recomendada = random.choice(canciones_disponibles['Song_name'].values)
        #Si esta le devuelvo otra random del top
        return f"Te recomiendo escuchar '{cancion_recomendada}' de {top_songs.loc[top_songs['Song_name'] == cancion_recomendada, 'Artist_name'].values[0]}."
    else:
        #si no esta la busco en spotify
        results = sp.search(q=cancion_ingresada, limit=50)
        tracks = json_normalize(results["tracks"]["items"])
        #creo un dataframe para poder ver todos los artistas
        def expand_list_dict(row):
            df = json_normalize(row['artists'])
            df['song_id'] = row['id']
            return df
        #llamo a mi funcion para separar los artistas
        tracks['artists_dfs'] = tracks.apply(expand_list_dict, axis=1)
        artist_df = pd.DataFrame(columns=['external_urls.spotify', 'href', 'id', 'name', 'type', 'uri', 'song_id'])
        #Uno mi nuevo data frame con mi viejo
        for mini_df in tracks['artists_dfs']:
            artist_df = pd.concat([artist_df, mini_df], axis=0)
        #Uno todo junto
        df_merged = pd.merge(left=tracks,
                    right=artist_df,
                    how='inner',
                    left_on='id',
                    right_on='song_id')
        #Me quedo solo con las columnas que necesito
        df_final = df_merged[['name_x', 'name_y', 'song_id']]
        df_final.columns = ['Song_name', 'Artist_name', 'song_id']
 
        cancioningre = pd.DataFrame(columns=['Song_name', 'Artist_name', 'song_id'])
        # Iterar sobre las filas 
        for index, row in df_final.iterrows():
            row_confirmation = input(f"¿Es esta tu canción? '{row['Song_name']}' de {row['Artist_name']} (yes/no): ").lower()
        # Si la respuesta es sí, agregar la fila al nuevo DataFrame y salir del bucle
            if row_confirmation == 'yes':
                cancioningre = cancioningre.append(row, ignore_index=True)
                break
            elif row_confirmation != 'no':
                print("Respuesta no válida. Por favor, responde 'yes' o 'no'.")

        cancionfeatures = sp.audio_features(cancioningre['song_id'])
        cancionfeaturesdf = json_normalize(cancionfeatures)
        cancionfinal = pd.merge(left=cancioningre,
                        right=cancionfeaturesdf,
                        how='inner',
                        left_on='song_id',
                        right_on='id')  
        #Ahora quiero escalar y sacar mi cluster
        cancionfinal2 = cancionfinal.select_dtypes(include=['number'])
        cancionfinal2_esc = dataspotify_sca.transform(cancionfinal2)
        clusters = kmeans.predict(cancionfinal2_esc)
        pd.Series(clusters).value_counts().sort_index()
        cancionfinal['Clusters']=clusters
        cancionfinal_cluster = cancionfinal[['Song_name','Artist_name','Clusters']]
        
        onlycluster = cancionfinal_cluster['Clusters'].values[0]
        # Voy a filtrar y me quedo solo con las canciones con el mismo cluster
        onlysongs = data_clusters[data_clusters['Clusters'] == onlycluster]       
        cancion_recomendada = random.choice(onlysongs['Song_name'].values)
        return f"Te recomiendo escuchar '{cancion_recomendada}' de {onlysongs.loc[onlysongs['Song_name'] == cancion_recomendada, 'Artist_name'].values[0]}."

cancion_ingresada = input("Ingresa el nombre de una canción: ")
resultado = recomendar(top_songs, cancion_ingresada, data_clusters)
print(resultado)

Ingresa el nombre de una canción: I can't fell my face
¿Es esta tu canción? 'I Cant Feel My Face' de GOTOU寄生獣 (yes/no): yes


/var/folders/d9/_ctzl6l91nb3xf6zfxnk4py80000gn/T/ipykernel_28651/83482592.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cancioningre = cancioningre.append(row, ignore_index=True)


Te recomiendo escuchar 'near dark' de Burial.
